https://www.upgrad.com/blog/association-rule-mining-an-overview-and-its-applications/

In [1]:
import os 

os.chdir("/home/hamza-arain/Documents/code/recmmendation")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.expand_frame_repr', None)

In [4]:
# df = pd.read_excel('data/Online Retail.xlsx')
df = pd.read_csv("data/online_retail_final.csv")
df.shape
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,404090.0,9.328724,174.542052,-80995.000,2.00,5.00,12.00,147.0
UnitPrice,404090.0,3.474446,69.549369,0.001,1.25,1.95,3.75,38970.0
CustomerID,404090.0,15286.660407,1712.698993,12346.000,13954.00,15152.00,16791.00,18287.0
TotalPrice,404090.0,16.854000,308.731606,-168469.600,4.20,10.50,18.72,38970.0


In [6]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
TotalPrice     0
dtype: int64

In [7]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile3 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe['InvoiceNo'].str.contains("C", na=False)]
    dataframe = dataframe[~dataframe['StockCode'].str.contains("POST", na=False)]
    dataframe = dataframe[~dataframe['StockCode'].str.contains("C2", na=False)]
    dataframe = dataframe[~dataframe['StockCode'].str.contains("M", na=False)]

    dataframe = dataframe[dataframe['Quantity'] > 0]
    dataframe = dataframe[dataframe['UnitPrice'] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "UnitPrice")
    return dataframe

df['InvoiceNo'] = df['InvoiceNo'].astype(str)
df = retail_data_prep(df)

/tmp/ipykernel_11866/780506749.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-46.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit


In [8]:
df.isnull().sum()
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,393376.0,10.261333,16.241236,1.000,2.00,5.00,12.00,147.00
UnitPrice,393376.0,2.832370,2.920073,0.001,1.25,1.95,3.75,31.44
CustomerID,393376.0,15300.349866,1709.035506,12347.000,13975.00,15159.00,16800.00,18287.00
TotalPrice,393376.0,18.512494,70.764231,0.001,4.68,11.70,19.50,38970.00


In [39]:
df_fr = df[df['Country'] == "France"]
df_fr.shape
# (8342, 8)

(8012, 9)

In [40]:
df_fr.groupby(['InvoiceNo', 'StockCode']). \
    agg({"Quantity": "sum"}). \
    unstack(). \
    fillna(0). \
    applymap(lambda x:1 if x>0 else 0).iloc[0:5, 0:5]

/tmp/ipykernel_11866/2912792920.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  applymap(lambda x:1 if x>0 else 0).iloc[0:5, 0:5]


Quantity                        
StockCode    10002 10120 10125 10135 11001
InvoiceNo                                 
536370           1     0     0     0     0
536852           0     0     0     0     0
536974           0     0     0     0     0
537065           0     0     0     0     0
537463           0     0     0     0     0

In [41]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['InvoiceNo', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)


fr_inv_pro_df = create_invoice_product_df(df_fr, id=True)
fr_inv_pro_df

/tmp/ipykernel_11866/1700227471.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  applymap(lambda x: 1 if x > 0 else 0)


StockCode  10002  10120  10125  10135  11001  15036  15039  15044C  15056BL  \
InvoiceNo                                                                     
536370         1      0      0      0      0      0      0       0        0   
536852         0      0      0      0      0      0      0       0        0   
536974         0      0      0      0      0      0      0       0        1   
537065         0      0      0      0      0      0      0       0        0   
537463         0      0      0      0      0      0      0       0        0   
...          ...    ...    ...    ...    ...    ...    ...     ...      ...   
580986         0      0      0      0      0      0      0       0        0   
581001         0      0      0      0      0      0      0       0        0   
581171         0      0      0      0      0      0      0       0        0   
581279         0      0      0      0      0      0      0       0        0   
581587         0      0      0      0      0      0      0       0        0   

StockCode  15056N  15056P  15058A  15058B  15058C  16012  16048  16156L  \
InvoiceNo                                                                 
536370          0       0       0       0       0      0      0       0   
536852          0       0       0       0       0      0      0       0   
536974          0       1       0       0       0      0      0       0   
537065          0       0       0       0       0      0      0       0   
537463          0       0       0       0       0      0      0       0   
...           ...     ...     ...     ...     ...    ...    ...     ...   
580986          0       0       0       0       0      0      0       0   
581001          0       0       0       0       0      0      0       0   
581171          0       0       0       0       0      0      0       0   
581279          0       0       0       0       0      0      0       0   
581587          0       0       0       0       0      0      0       0   

StockCode  16156S  16161P  16161U  16169E  16218  16219  16225  16236  16237  \
InvoiceNo                                                                      
536370          0       0       0       0      0      0      0      0      0   
536852          0       0       0       0      0      0      0      0      0   
536974          0       0       0       0      0      0      0      0      0   
537065          0       0       0       0      0      0      0      0      0   
537463          0       0       0       0      0      0      0      0      0   
...           ...     ...     ...     ...    ...    ...    ...    ...    ...   
580986          0       0       0       0      0      0      0      0      0   
581001          0       0       0       0      0      0      0      0      0   
581171          0       0       0       0      0      0      0      0      0   
581279          0       0       0       0      0      0      0      0      0   
581587          0       0       0       0      0      0      0      0      0   

StockCode  16238  16258A  17011F  17012A  17012B  17174  20615  20617  20658  \
InvoiceNo                                                                      
536370         0       0       0       0       0      0      0      0      0   
536852         0       0       0       0       0      0      0      0      0   
536974         0       0       0       0       0      0      0      0      0   
537065         1       0       0       0       0      0      0      0      0   
537463         0       0       0       0       0      0      0      0      0   
...          ...     ...     ...     ...     ...    ...    ...    ...    ...   
580986         0       0       0       0       0      0      0      0      0   
581001         0       0       0       0       0      0      0      0      0   
581171         0       0       0       0       0      0      0      0      0   
581279         0       0       0       0       0      0      0      0      

In [42]:
# converting all positive vaues to 1 and everything else to 0
def my_encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

fr_inv_pro_df = fr_inv_pro_df.applymap(my_encode_units)
fr_inv_pro_df

/tmp/ipykernel_11866/2263106245.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fr_inv_pro_df = fr_inv_pro_df.applymap(my_encode_units)


StockCode  10002  10120  10125  10135  11001  15036  15039  15044C  15056BL  \
InvoiceNo                                                                     
536370         1      0      0      0      0      0      0       0        0   
536852         0      0      0      0      0      0      0       0        0   
536974         0      0      0      0      0      0      0       0        1   
537065         0      0      0      0      0      0      0       0        0   
537463         0      0      0      0      0      0      0       0        0   
...          ...    ...    ...    ...    ...    ...    ...     ...      ...   
580986         0      0      0      0      0      0      0       0        0   
581001         0      0      0      0      0      0      0       0        0   
581171         0      0      0      0      0      0      0       0        0   
581279         0      0      0      0      0      0      0       0        0   
581587         0      0      0      0      0      0      0       0        0   

StockCode  15056N  15056P  15058A  15058B  15058C  16012  16048  16156L  \
InvoiceNo                                                                 
536370          0       0       0       0       0      0      0       0   
536852          0       0       0       0       0      0      0       0   
536974          0       1       0       0       0      0      0       0   
537065          0       0       0       0       0      0      0       0   
537463          0       0       0       0       0      0      0       0   
...           ...     ...     ...     ...     ...    ...    ...     ...   
580986          0       0       0       0       0      0      0       0   
581001          0       0       0       0       0      0      0       0   
581171          0       0       0       0       0      0      0       0   
581279          0       0       0       0       0      0      0       0   
581587          0       0       0       0       0      0      0       0   

StockCode  16156S  16161P  16161U  16169E  16218  16219  16225  16236  16237  \
InvoiceNo                                                                      
536370          0       0       0       0      0      0      0      0      0   
536852          0       0       0       0      0      0      0      0      0   
536974          0       0       0       0      0      0      0      0      0   
537065          0       0       0       0      0      0      0      0      0   
537463          0       0       0       0      0      0      0      0      0   
...           ...     ...     ...     ...    ...    ...    ...    ...    ...   
580986          0       0       0       0      0      0      0      0      0   
581001          0       0       0       0      0      0      0      0      0   
581171          0       0       0       0      0      0      0      0      0   
581279          0       0       0       0      0      0      0      0      0   
581587          0       0       0       0      0      0      0      0      0   

StockCode  16238  16258A  17011F  17012A  17012B  17174  20615  20617  20658  \
InvoiceNo                                                                      
536370         0       0       0       0       0      0      0      0      0   
536852         0       0       0       0       0      0      0      0      0   
536974         0       0       0       0       0      0      0      0      0   
537065         1       0       0       0       0      0      0      0      0   
537463         0       0       0       0       0      0      0      0      0   
...          ...     ...     ...     ...     ...    ...    ...    ...    ...   
580986         0       0       0       0       0      0      0      0      0   
581001         0       0       0       0       0      0      0      0      0   
581171         0       0       0       0       0      0      0      0      0   
581279         0       0       0       0       0      0      0      0      

In [43]:
fr_inv_pro_df.columns

Index(['10002', '10120', '10125', '10135', '11001', '15036', '15039', '15044C',
       '15056BL', '15056N',
       ...
       '85230G', '85232D', '90030B', '90030C', '90031', '90099', '90184B',
       '90184C', '90201B', '90201C'],
      dtype='object', name='StockCode', length=1517)

In [44]:
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)

check_id(df_fr, '10002')

['INFLATABLE POLITICAL GLOBE ']


In [45]:
frequent_itemsets = apriori(fr_inv_pro_df,
                            min_support=0.07,
                            use_colnames=True)

frequent_itemsets.sort_values("support", ascending=False)

/home/hamza-arain/Documents/code/recmmendation/.venv/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
14,0.185185,(21731)
43,0.185185,(23084)
30,0.177249,(22554)
32,0.174603,(22556)
23,0.164021,(22326)
...,...,...
37,0.071429,(22659)
36,0.071429,(22631)
44,0.071429,(23203)
49,0.071429,(23256)


Support: Support represents the percentage of items seen together in the association rule. For example, if the association rule has a support value of 5%, it means that 5% of the related items are seen together.</br>
Confidence: Reliability refers to how often the association rule is true. If the concurrency value of the association rule is 80%, it means that the rule is true when 80% of the related items are seen together.</br>
Win (Lift): Win shows how “interesting” the association rule is. If the lift value is greater than 1, it indicates that the association rule is above the predicted dependency and there is an interesting relationship.</br>

In [46]:
rules = association_rules(frequent_itemsets,
                  metric="support",
                  min_threshold=0.01,
                  num_itemsets=len(frequent_itemsets))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(21080),(21086),0.137566,0.142857,0.105820,0.769231,5.384615,1.0,0.086168,3.714286,0.944172,0.606061,0.730769,0.754986
1,(21086),(21080),0.142857,0.137566,0.105820,0.740741,5.384615,1.0,0.086168,3.326531,0.950000,0.606061,0.699387,0.754986
2,(21080),(21094),0.137566,0.132275,0.105820,0.769231,5.815385,1.0,0.087624,3.760141,0.960123,0.645161,0.734053,0.784615
3,(21094),(21080),0.132275,0.137566,0.105820,0.800000,5.815385,1.0,0.087624,4.312169,0.954268,0.645161,0.768098,0.784615
4,(21094),(21086),0.132275,0.142857,0.126984,0.960000,6.720000,1.0,0.108088,21.428571,0.980945,0.857143,0.953333,0.924444
5,(21086),(21094),0.142857,0.132275,0.126984,0.888889,6.720000,1.0,0.108088,7.809524,0.993056,0.857143,0.871951,0.924444
6,(22554),(22551),0.177249,0.140212,0.108466,0.611940,4.364404,1.0,0.083613,2.215608,0.936946,0.518987,0.548657,0.692763
7,(22551),(22554),0.140212,0.177249,0.108466,0.773585,4.364404,1.0,0.083613,3.633818,0.896585,0.518987,0.724807,0.692763
8,(22556),(22551),0.174603,0.140212,0.092593,0.530303,3.782161,1.0,0.068111,1.830517,0.891209,0.416667,0.453706,0.595340
9,(22551),(22556),0.140212,0.174603,0.092593,0.660377,3.782161,1.0,0.068111,2.430335,0.855560,0.416667,0.588534,0.595340


<br>Antecedents: Antecedents refer to the items or events that are present before a particular event or rule. They represent the input or condition in an association rule.</br>
<br>Consequents: Consequents are the items or events that follow a specific event or rule. They represent the output or result in an association rule.</br>
<br>Support: Support is a measure that indicates the frequency or proportion of transactions that contain both the antecedent and consequent items. It shows how often the items appear together in the dataset.</br>
<br>Antecedent support: Antecedent support refers to the proportion of transactions in the dataset that contain the antecedent item.</br>
<br>Consequent support: Consequent support represents the proportion of transactions in the dataset that contain the consequent item.</br>
<br>Lift: Lift is a measure of the strength of an association rule. It indicates how much more likely the consequent item is to be purchased when the antecedent item is present, compared to its individual occurrence rate. A lift value greater than 1 indicates a positive relationship.</br>
<br>Leverage: Leverage quantifies the difference between the observed frequency of the antecedent and consequent items appearing together and the frequency that would be expected if they were independent. It measures the significance of the association rule.</br>
<br>Confidence: Confidence is a measure of the reliability or certainty of an association rule. It represents the conditional probability of the consequent item given the antecedent item. A higher confidence value indicates a stronger predictive power of the rule.</br>

In [47]:
rules[(rules["support"]>0.05) & (rules["confidence"]>0.1) & (rules["lift"]>5)].sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
21,"(21080, 21086)",(21094),0.105820,0.132275,0.103175,0.975000,7.371000,1.0,0.089177,34.708995,0.966621,0.764706,0.971189,0.877500
20,"(21080, 21094)",(21086),0.105820,0.142857,0.103175,0.975000,6.825000,1.0,0.088057,34.285714,0.954483,0.709091,0.970833,0.848611
4,(21094),(21086),0.132275,0.142857,0.126984,0.960000,6.720000,1.0,0.108088,21.428571,0.980945,0.857143,0.953333,0.924444
5,(21086),(21094),0.142857,0.132275,0.126984,0.888889,6.720000,1.0,0.108088,7.809524,0.993056,0.857143,0.871951,0.924444
14,(22727),(22726),0.097884,0.100529,0.082011,0.837838,8.334282,1.0,0.072170,5.546737,0.975499,0.704545,0.819714,0.826814
15,(22726),(22727),0.100529,0.097884,0.082011,0.815789,8.334282,1.0,0.072170,4.897203,0.978368,0.704545,0.795802,0.826814
22,"(21094, 21086)",(21080),0.126984,0.137566,0.103175,0.812500,5.906250,1.0,0.085706,4.599647,0.951515,0.639344,0.782592,0.781250
3,(21094),(21080),0.132275,0.137566,0.105820,0.800000,5.815385,1.0,0.087624,4.312169,0.954268,0.645161,0.768098,0.784615
19,(22727),(22728),0.097884,0.105820,0.076720,0.783784,7.406757,1.0,0.066362,4.135582,0.958843,0.604167,0.758196,0.754392
24,(21094),"(21080, 21086)",0.132275,0.105820,0.103175,0.780000,7.371000,1.0,0.089177,4.064454,0.996091,0.764706,0.753964,0.877500


We can make the following comment here, it should be noted that we are considering it in the context of confidence, when 21080 and 21094 are taken, the probability of getting 21086 is 0.97, very high.

In [50]:
product_id = '21086'
check_id(df, product_id)
# ['MINI PAINT SET VINTAGE ']

['SET/6 RED SPOTTY PAPER CUPS']


In [51]:
sorted_rules = rules.sort_values("lift", ascending=False)

recommendation_list = []

for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

recommendation_list[0:3]
# [22556, 22551, 22326]

['21094', '21080', '21094']

In [52]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])
    return recommendation_list

# arl_recommender(rules, 22492, 2)
# [22556, 22551]
arl_recommender(rules, '21080', 3)
# [22556, 22551, 22326]

['21094', '21086', '21094', '21094', '21086']

In conclusion, Association Rule Learning is a powerful technique that uncovers associations and patterns in data, particularly in market basket analysis. It offers advantages such as revealing hidden relationships, personalizing marketing efforts, and providing easily interpretable rules. However, it also has limitations in terms of rule quantity and the lack of temporal or causal insights. Understanding these aspects is crucial for leveraging the benefits of Association Rule Learning in practical applications.

If you would like to see more content related to my journey in data science, you can follow me on the following platforms.